In [1]:
import cv2
import numpy as np

# 예제 9.6 ORB 특징 검출 및 디스크립터
### <span style='background-color:#fff5b1'>cv2.ORB</span>
**create**
**detect**
**detectAndCompute()**

In [2]:
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x1-x2) ** 2 + (y1-y2) ** 2)

In [3]:
def filteringByDistance(kp, distE = 0.5):
    size = len(kp)
    mask = np.arange(1, size + 1).astype(np.bool_)
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
    print("False 개수", list(mask).count(False), "-> 필터링을 통해서 제거되는 pt의 개수")
    np_kp = np.array(kp)
    return list(np_kp[mask])

In [4]:
src = np.zeros(shape = (512,512,3), dtype = np.uint8) + 255
cv2.rectangle(src, (100,100), (300,300), (0,0,0), -1)
cv2.rectangle(src, (200,200), (400,400), (0,0,0), -1)

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
cv2.GaussianBlur(gray, (5,5), 0.0)

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [5]:
orbF = cv2.ORB_create(scoreType = 1) # Fast scroe로 반응값 계산 <-> Harris score
kp = orbF.detect(gray)
print(len(kp))
dst = cv2.drawKeypoints(gray, kp, None, color = (0,0,255))

49


In [6]:
kp = sorted(kp, key=lambda f: f.response, reverse = True)
filtered_kp = list(filter(lambda f: f.response > 50, kp))
filtered_kp = filteringByDistance(kp, 10)
len(filtered_kp)

False 개수 41 -> 필터링을 통해서 제거되는 pt의 개수


8

In [7]:
kp, des = orbF.compute(gray, filtered_kp) # 디스크립터 계산
print(des.shape)
for i in des:
    # for j in i:
    #     print(j, end=" ")
    # print()
    print(i)

(8, 32)
[  2 141   2 158  20 131 170 230  22   0  88   2  32 234  48  64  89 106
  19 129   2 184   8 255  32  38 138 128  20  10  48 136]
[  2 141   2 158  20 131 170 230  22   0  90   2  96 234  48  64  89 106
  19 129  34 152  10 255  32  38 138 128  20  10  48 136]
[  2 141   2 158  20 131 170 230  22   1  88   2  96 232  48  64  89 106
  19 129  34 184   8 255  32  38 138 136  20  10  48 136]
[  2 141   2 158  20 131 170 230  22   0  88   2  32 234  48  64  89 106
  19 129   2 184   8 255  32  38 138 128  20  10  48 136]
[  2 141   2 158  20 131 170 230  22   1  88   2  96 232  48  64  89 106
  19 129  34 184   8 255  32  38 138 136  20  10  48 136]
[  2 141   2 158  20 131 170 230  22   0  90   2  96 234  48  64  89 106
  19 129  34 152  10 255  32  38 138 128  20  10  48 136]
[104  48  56  96  32  76  81  16  97   8  52   8 159  23   0  48 132  20
  93   8 136  96 176   0 195 249  32  16  66 128  70  51]
[104  48  56  96  33  77  81  16 105 136  52   8 159  23   0  48 132  21
  

In [8]:
dst2 = cv2.drawKeypoints(gray, filtered_kp, None, color = (0,0,255))
for f in filtered_kp:
    x, y = f.pt
    size = f.size
    rect = ((x,y), (size,size), f.angle)
    box = cv2.boxPoints(rect).astype(np.int32)
    cv2.polylines(dst2, [box], True, (0,255,0), 2)
    cv2.circle(dst2, (round(x), round(y)), round(f.size / 2), (255,0,0), 2)

In [9]:
cv2.imshow('dst', dst)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

# 예제 9.7 다른 특징 검출기의 ORB 디스크립터

In [10]:
src = np.zeros(shape = (512,512,3), dtype = np.uint8) + 255
cv2.rectangle(src, (100,100), (300,300), (0,0,0), -1)
cv2.rectangle(src, (200,200), (400,400), (0,0,0), -1)

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

In [11]:
fastF = cv2.FastFeatureDetector_create(threshold=10)
# mserF = cv2.MSER_create(10)
# blobF = cv2.SimpleBlobDetector_create()
# goodF = cv2.GFTTDetector_create(maxCorners = 20, minDistance = 10)

In [12]:
kp = fastF.detect(gray)
# kp = mserF.detect(gray)
# kp = blobF.detect(gray)
# kp = goodF.detect(gray)
len(kp)

0

In [13]:
filtered_kp = filteringByDistance(kp, 10)
print(len(filtered_kp))
dst  = cv2.drawKeypoints(gray, filtered_kp, None, color = (0,0,255))

False 개수 0 -> 필터링을 통해서 제거되는 pt의 개수
0


In [14]:
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()